# Import des modules

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from ast import literal_eval
import csv
import re

# Création des variables

In [2]:
DF0 = pd.read_csv("./ReferentielNoms.csv",converters={"FormesNonNormalisées": literal_eval}).set_index("NomNormalisé")
DF0

,Types,FormesNonNormalisées,RenvoiToponymique
NomNormalisé,,,
"aarau, von",['toponymic'],"[von arowe, aarau, von]",Aarau
achenheim,['toponymic'],[achenheim],Achenheim
"achenheim, von",['toponymic'],"[von achenheim, achenheim, von]",Achenheim
achtjahre,['cognominal'],"[acht jore, achtjahre]",NaN
acker,['cognominal'],"[acker, ackel]",NaN
...,...,...,...
ölmann,['occupational'],"[oleman, ölmann, olman]",NaN
ösenwoog,['toponymic'],"[ösenwog, ösenwoog]",NaN
österreicher,['toponymic'],"[estericher, österreicher]",Autriche


# Création de la soupe

In [3]:
soup = BeautifulSoup(open('./../Sources/EditionsXML/aves_4R_p131-220.xml', 'r'))

# Création de la liste des éléments de la soupe à comparer

In [4]:
ListeNomsDoc = []
for surname in soup.find_all('surname'):
    surname = surname.get_text().lower().replace('\n', ' ')
    surname = re.sub(' +',' ',surname)
    ListeNomsDoc.append(surname)
for addName in soup.find_all('addname'):
    addName = addName.get_text().lower().replace('\n', ' ')
    addName = re.sub(' +',' ',addName)
    ListeNomsDoc.append(addName)
print (ListeNomsDoc)

['von kageneck', 'von mulnheim', 'löselin', 'klobelouch', 'metziger', 'marsilius', 'von mülnheim', 'von endingen', 'hüffelin', 'zorn', 'von mülnheim', 'hüffelin', 'zorn', 'bock', 'hesse', 'merswin', 'tütschman', 'rippelin', 'von heiligenstein', 'gürteler', 'bühssener', 'züm trübel', 'sturm', 'züm störcken', 'göudertheim', 'völtsche', 'von wintertur', 'rebstock', 'nope', 'von colmar', 'gerung', 'rotschilt', 'spilman', 'schott', 'isenhusen', 'kindelin', 'von ginebreht', 'boppe', 'tumeler', 'westerman', 'heischer', 'loubeck', 'honowe', 'hünrer', 'nattener', 'swicker', 'aler', 'gosse', 'von ingenheim', 'diether', 'dingesheim', 'von hagenöwe', 'stahel', 'klye', 'rynöwe', 'von wickersheim', 'bock', 'von heiligenstein', 'von rosheim', 'sünner', 'zorn', 'von mülnheim', 'swarber', 'burggraf', 'wetzel', 'von mülnheim', 'maler', 'von matzenheim', 'von stille', 'von endingen', 'zür megde', 'berlin', 'gürtler', 'blümenowe', 'rebstock', 'von cünheim', 'mansse', 'bock', 'hesse', 'kloblouch', 'ellehar

# Comparaison de la liste de la soupe à notre dataframe

In [5]:
for element in ListeNomsDoc :
    
    #Création des listes de références à partir du dataframe DF1.
    Colonne_FormesNonNormalisées = DF0['FormesNonNormalisées'].tolist()
    Colonne_NomNormalisé = DF0.index.tolist()
    Colonne_Types = DF0['Types'].tolist()
    
    #Vérification de l'existence de la forme non normalisée dans la liste des listes de formes non-normalisées.
    CheckSurnameNN = any(element in sublist for sublist in Colonne_FormesNonNormalisées)
    DicoPersonne = {}

    #Sinon, on demande un str en entrée pour le comparer à la liiste des formes normalisées.
    if CheckSurnameNN != True :
        SurnameN = input('Entrez la forme normalisée du nom ' + element + ' :')
        
        #Si l'entrée figure dans la liste des prénoms normalisées, on ajoute la forme non normalisée à la liste de celles-ci dans le dataframe.
        if SurnameN in Colonne_NomNormalisé:
            CelluleModif = DF0.loc[SurnameN, 'FormesNonNormalisées']
            CelluleModif.append(element)
            
        #Sinon, on crée un dataframe qu'on additionnera à l'existant.
        else :
            
            #On crée le couple 'PrénomNormalisé' dont la valeur correspond à la forme la plus courante du prénom.
            DicoPersonne['NomNormalisé'] = SurnameN
            
            #On crée le couple 'Types' dont la valeur est une liste contenant des tags proposant une description typologique du prénom.
            typologie = []
            n = int(input('Entrez le nombre de types à attribuer au nom ' + element + ' :'))
            for i in range(0,n):
                Type = input('Entrez un des types à attribuer au nom ' + element + ' :')
                typologie.append(Type)
            DicoPersonne['Types'] = typologie
            
            #On crée le couple 'FormesNonNormalisées' dont la valeur est une liste contenant toutes les formes alternatives du prénom.
            ListeNN = []
            ListeNN.append(element)
            if element != SurnameN :
                ListeNN.append(SurnameN)
            DicoPersonne['FormesNonNormalisées'] = ListeNN
            
            #On crée le couple 'RenvoiToponymique' dont la valeur est le nom normalisé en français (si possible) du lieu.
            for Type in typologie:
                if Type == 'toponymic':
                    RenvoiToponymique = input('Entrez le nom normalisé, en français si possible, du lieu auquel' + element + ' fait référence : ')
                    DicoPersonne['RenvoiToponymique'] = RenvoiToponymique
            
            #On crée le dataframe à ajouter à l'existant.
            DF1 = pd.DataFrame([DicoPersonne]).set_index('NomNormalisé')
            
            #On concatène le dictionnaire obtenu à notre Referentiel avant de recommencer toute l'opération.
            DF0 = pd.concat([DF0,DF1])
DF0

Entrez la forme normalisée du nom zuͦr wilgen :wilgen, zur
Entrez la forme normalisée du nom zuͦr kenen :kenen, zur
Entrez la forme normalisée du nom zuͦr kennen :kenen, zur
Entrez la forme normalisée du nom zuͦr hellen :hellen, zur
Entrez la forme normalisée du nom zuͦr meͣgde :megde, zur
Entrez la forme normalisée du nom windeck :windeck
Entrez le nombre de types à attribuer au nom windeck :1
Entrez un des types à attribuer au nom windeck :toponymic
Entrez le nom normalisé, en français si possible, du lieu auquelwindeck fait référence : 
Entrez la forme normalisée du nom ackenfuͦs :ackerfuss
Entrez la forme normalisée du nom sant martin :sankt-martin
Entrez le nombre de types à attribuer au nom sant martin :1
Entrez un des types à attribuer au nom sant martin :toponymic
Entrez le nom normalisé, en français si possible, du lieu auquelsant martin fait référence : 


,Types,FormesNonNormalisées,RenvoiToponymique
NomNormalisé,,,
"aarau, von",['toponymic'],"[von arowe, aarau, von]",Aarau
achenheim,['toponymic'],[achenheim],Achenheim
"achenheim, von",['toponymic'],"[von achenheim, achenheim, von]",Achenheim
achtjahre,['cognominal'],"[acht jore, achtjahre]",NaN
acker,['cognominal'],"[acker, ackel]",NaN
...,...,...,...
österreicher,['toponymic'],"[estericher, österreicher]",Autriche
"überhang, zum",[],"[zum überhange, überhang, zum, züm überhang, z...",NaN
üsenberg,['toponymic'],"[uͦsemberg, üsemberg, üsenberg, usenberg, usen...",Üsenberg


In [6]:
DF0 = DF0.sort_values(by=['NomNormalisé'])
DF0.to_csv("./ReferentielNoms.csv")